In [9]:
%pip install mcp jupyter-server-proxy openai python-dotenv nest_asyncio ollama requests
%pip install langchain langchain-core langchain-community langchain-ollama
%pip install -U transformers sentencepiece torch
%pip install ollmcp fastapi uvicorn httpx pydantic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached mcp-1.12.4-py3-none-any.whl.metadata (68 kB)
Using cached mcp-1.12.4-py3-none-any.whl (160 kB)
  Attempting uninstall: mcp
    Found existing installation: mcp 1.15.0
    Uninstalling mcp-1.15.0:
      Successfully uninstalled mcp-1.15.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Environment Setup
Create a .env file in project root with:

OPENWEATHER_API_KEY=YOUR_KEY_HERE
OPENWEATHER_BASE_URL=https://api.openweathermap.org/data/2.5/weather
OPENAI_API_KEY=YOUR_OPENAI_KEY

If only OPEN_AI_API_KEY exists it will be aliased automatically.
Restart the kernel after changes.

In [10]:
import os
import json
from typing import List, Dict, Any
import requests
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env if present

def ensure_env(name: str, default: str | None = None, prompt: bool = True, secret: bool = False) -> str:
    val = os.getenv(name)
    if val:
        return val
    if default is not None:
        os.environ[name] = default
        return default
    if prompt:
        try:
            entered = input(f"Enter value for {name}: ").strip()
            if entered:
                os.environ[name] = entered
                return entered
        except Exception:
            pass
    raise RuntimeError(f"{name} is not set")

In [11]:
def get_current_weather(location: str, unit: str = "celsius") -> Dict[str, Any]:
    """Fetch current weather from OpenWeather."""
    key = ensure_env("OPENWEATHER_API_KEY", prompt=True)
    base_url = os.getenv("OPENWEATHER_BASE_URL") or "https://api.openweathermap.org/data/2.5/weather"

    unit_map = {"celsius": "metric", "fahrenheit": "imperial"}
    owm_unit = unit_map.get(unit.lower(), "metric")
    params = {"q": location, "units": owm_unit, "appid": key}
    resp = requests.get(base_url, params=params, timeout=15)
    resp.raise_for_status()
    data = resp.json()
    resolved_name = data.get("name") or location
    temp = (data.get("main") or {}).get("temp")
    weather_list = data.get("weather")
    forecast = [w.get("description") for w in weather_list if isinstance(w, dict) and w.get("description")] if isinstance(weather_list, list) else []
    return {
        "location": resolved_name,
        "temperature": temp,
        "unit": "celsius" if owm_unit == "metric" else "fahrenheit",
        "forecast": forecast
    }
# get_current_weather("Milan")

In [12]:
# OpenAI tool schemas (JSON Schema)
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get current weather for a location using OpenWeather. Units can be 'celsius' or 'fahrenheit'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name (optionally with country code)"},
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "default": "celsius",
                        "description": "Temperature unit"
                    }
                },
                "required": ["location"],
                "additionalProperties": False
            }
        }
    }
]

In [13]:
# Map tool names to Python callables
mapping_tool_function = {
    "get_current_weather": get_current_weather
}

def execute_tool(tool_name: str, tool_args: Dict[str, Any]) -> str:
    result = mapping_tool_function[tool_name](**tool_args)
    if result is None:
        return "The operation completed but didn't return any results."
    if isinstance(result, list):
        return ", ".join(map(str, result))
    if isinstance(result, dict):
        return json.dumps(result, indent=2)
    return str(result)

In [14]:
# ----------------- OpenAI chat with tool calling -----------------
from openai import OpenAI


api_key = os.getenv("OPENAI_API_KEY")
BASE_OPENAI_URL = os.getenv("BASE_OPENAI_URL", "https://api.openai.com/v1")
client = OpenAI(api_key=api_key, base_url=BASE_OPENAI_URL)
OPENAI_MODEL = "gpt-3.5-turbo"
print(f"Using OpenAI model: {OPENAI_MODEL}")

def process_query(query: str) -> None:
    messages: List[Dict[str, Any]] = [{"role": "user", "content": query}]
    response = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        temperature=0.01
    )
    while True:
        choice = response.choices[0]
        msg = choice.message
        if msg.tool_calls:
            messages.append({
                "role": "assistant",
                "content": msg.content or "",
                "tool_calls": [tc.model_dump() for tc in msg.tool_calls]
            })
            for tc in msg.tool_calls:
                name = tc.function.name
                args = json.loads(tc.function.arguments or "{}")
                result = execute_tool(name, args)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": result
                })
            response = client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=messages,
                tools=tools,
                tool_choice="auto",
                temperature=0.01
            )
            continue
        if msg.content:
            print(msg.content)
        break
    
process_query("What's the weather like in New York and Milan?")

Using OpenAI model: gpt-3.5-turbo
The current weather in New York is 9.43°C with clear skies, and in Milan, it is 21.44°C with scattered clouds.


In [15]:
api_key = os.getenv("OPENAI_API_KEY")
import os
!curl -s https://api.openai.com/v1/models -H "Authorization: Bearer {os.environ['OPENAI_API_KEY']}"


{
  "object": "list",
  "data": [
    {
      "id": "gpt-4-0613",
      "object": "model",
      "created": 1686588896,
      "owned_by": "openai"
    },
    {
      "id": "gpt-4",
      "object": "model",
      "created": 1687882411,
      "owned_by": "openai"
    },
    {
      "id": "gpt-3.5-turbo",
      "object": "model",
      "created": 1677610602,
      "owned_by": "openai"
    },
    {
      "id": "sora-2-pro",
      "object": "model",
      "created": 1759708663,
      "owned_by": "system"
    },
    {
      "id": "gpt-audio-mini-2025-10-06",
      "object": "model",
      "created": 1759512137,
      "owned_by": "system"
    },
    {
      "id": "gpt-realtime-mini",
      "object": "model",
      "created": 1759517133,
      "owned_by": "system"
    },
    {
      "id": "gpt-realtime-mini-2025-10-06",
      "object": "model",
      "created": 1759517175,
      "owned_by": "system"
    },
    {
      "id": "sora-2",
      "object": "model",
      "created": 1759708615,
      

In [16]:
def chat_loop():
    print("Type your queries or 'quit' to exit.")
    while True:
        try:
            query = input("\nQuery: ").strip()
            if query.lower() == "quit":
                break
            process_query(query)
            print()
        except Exception as e:
            print(f"\nError: {str(e)}")

In [17]:
chat_loop()

Type your queries or 'quit' to exit.
Hello! How can I assist you today?



In [18]:
%%writefile weather_server.py
from mcp.server.fastmcp import FastMCP
import os, json, requests
from typing import Dict, Any
from dotenv import load_dotenv
load_dotenv()
mcp = FastMCP("weather")
def ensure_env(name: str, default: str | None = None) -> str:
    val = os.getenv(name)
    if val:
        return val
    if default is not None:
        os.environ[name] = default
        return default
    raise RuntimeError(f"{name} is not set")
@mcp.tool()
def get_current_weather(location: str, unit: str = "celsius") -> Dict[str, Any]:
    key = ensure_env("OPENWEATHER_API_KEY")
    base_url = os.getenv("OPENWEATHER_BASE_URL") or "https://api.openweathermap.org/data/2.5/weather"
    unit_map = {"celsius": "metric", "fahrenheit": "imperial"}
    owm_unit = unit_map.get(unit.lower(), "metric")
    params = {"q": location, "units": owm_unit, "appid": key}
    resp = requests.get(base_url, params=params, timeout=15)
    resp.raise_for_status()
    data = resp.json()
    resolved_name = data.get("name") or location
    temp = (data.get("main") or {}).get("temp")
    weather_list = data.get("weather")
    forecast = [w.get("description") for w in weather_list if isinstance(w, dict) and w.get("description")] if isinstance(weather_list, list) else []
    return {
        "location": resolved_name,
        "temperature": temp,
        "unit": "celsius" if owm_unit == "metric" else "fahrenheit",
        "forecast": forecast
    }
if __name__ == "__main__":
    mcp.run(transport='stdio')

Overwriting weather_server.py


In [19]:
%%writefile mcp_chatbot_openai.py
from dotenv import load_dotenv
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List, Dict, Any
import asyncio, json, os, nest_asyncio
load_dotenv()
nest_asyncio.apply()

def _flatten_tool_content(content_list):
    parts = []
    for item in content_list:
        # TextContent (MCP) typically has .text
        text = getattr(item, 'text', None)
        if text is not None:
            parts.append(text)
        elif isinstance(item, dict):
            parts.append(json.dumps(item, ensure_ascii=False))
        else:
            parts.append(str(item))
    return "\n".join(parts)

class MCP_ChatBot:
    def __init__(self):
        self.session: ClientSession | None = None
        api_key = os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self.available_tools: List[dict] = []

    def _openai_tools_from_mcp(self, mcp_tools: List[types.Tool]) -> List[dict]:
        out = []
        for t in mcp_tools:
            out.append({
                "type": "function",
                "function": {
                    "name": t.name,
                    "description": t.description or "",
                    "parameters": t.inputSchema or {"type": "object", "properties": {}}
                }
            })
        return out

    async def process_query(self, query: str, model: str = "gpt-3.5-turbo"):
        messages: List[Dict[str, Any]] = [{"role": "user", "content": query}]
        while True:
            response = self.client.chat.completions.create(
                model=model,
                messages=messages,
                tools=self.available_tools or None,
                tool_choice="auto" if self.available_tools else "none",
                temperature=0.01
            )
            msg = response.choices[0].message
            if msg.tool_calls:
                messages.append({
                    "role": "assistant",
                    "content": msg.content or "",
                    "tool_calls": [tc.model_dump() for tc in msg.tool_calls]
                })
                for tc in msg.tool_calls:
                    tool_name = tc.function.name
                    raw_args = tc.function.arguments
                    try:
                        args = json.loads(raw_args) if isinstance(raw_args, str) else (raw_args or {})
                    except Exception:
                        args = {}
                    result = await self.session.call_tool(tool_name, arguments=args)
                    flattened = _flatten_tool_content(result.content)
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tc.id,
                        "name": tool_name,
                        "content": flattened
                    })
                continue
            if msg.content:
                print(msg.content.strip())
            break

    async def chat_loop(self):
        print("MCP Chatbot Started. Type your queries or 'quit'.")
        while True:
            try:
                q = input("\nQuery: ").strip()
                if q.lower() == 'quit':
                    break
                await self.process_query(q)
            except Exception as e:
                print(f"Error: {e}")

    async def connect_to_server_and_run(self):
        server_params = StdioServerParameters(command="uv", args=["run", "weather_server.py"], env=None)
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                await session.initialize()
                resp = await session.list_tools()
                self.available_tools = self._openai_tools_from_mcp(resp.tools)
                await self.chat_loop()

async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()

if __name__ == '__main__':
    asyncio.run(main())

Overwriting mcp_chatbot_openai.py


In [ ]:
%%writefile mcp_chatbot_openai.py
from dotenv import load_dotenv
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List, Dict, TypedDict, Any, Union
from contextlib import AsyncExitStack
import asyncio, json, os

# ===== LangChain imports (with fallbacks for version differences) =====
try:
    from langchain_openai import ChatOpenAI  # >= 0.2 style
except Exception:
    from langchain.chat_models import ChatOpenAI  # legacy fallback

try:
    from langchain_community.chat_message_histories import FileChatMessageHistory
except Exception:
    # older versions
    from langchain.memory.chat_message_histories import FileChatMessageHistory  # type: ignore

from langchain.memory import ConversationSummaryBufferMemory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

load_dotenv()


def _flatten_tool_content(content_list) -> str:
    """Flatten MCP CallToolResult.content into plain text for OpenAI 'tool' message."""
    parts = []
    if isinstance(content_list, list):
        for item in content_list:
            t = getattr(item, "type", None) or (isinstance(item, dict) and item.get("type"))
            if t == "text":
                txt = getattr(item, "text", None) or (isinstance(item, dict) and item.get("text"))
                if txt is not None:
                    parts.append(str(txt))
            elif t in ("json", "object"):
                data = getattr(item, "data", None) or (isinstance(item, dict) and (item.get("data") or item.get("value")))
                try:
                    parts.append(json.dumps(data, ensure_ascii=False))
                except Exception:
                    parts.append(str(data))
            else:
                try:
                    parts.append(json.dumps(item, default=str, ensure_ascii=False))
                except Exception:
                    parts.append(str(item))
    elif content_list is not None:
        if isinstance(content_list, (dict, list)):
            parts.append(json.dumps(content_list, ensure_ascii=False))
        else:
            parts.append(str(content_list))
    return "\n".join(parts)


class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict


class MCP_ChatBot:
    def __init__(self):
        # --- OpenAI client for actual chat/tool-calling loop ---
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

        # --- MCP infra ---
        self.exit_stack = AsyncExitStack()
        self.sessions: List[ClientSession] = []
        self.tool_to_session: Dict[str, ClientSession] = {}
        self.available_tools: List[ToolDefinition] = []  # OpenAI function-calling schema

        # --- LangChain Memory (NEW) ---
        # Persistent history file (JSON). Change path via LC_HISTORY_PATH env var if you like.
        history_path = os.getenv("LC_HISTORY_PATH", ".mcp_chat_history.json")
        self.history = FileChatMessageHistory(history_path)

        # LLM used only for summarizing memory (cheap/small recommended)
        summary_model = os.getenv("OPENAI_SUMMARY_MODEL", "gpt-3.5-turbo")
        self.summary_llm = ChatOpenAI(
            model=summary_model,
            temperature=0.0,
            # LangChain uses OPENAI_API_KEY env under the hood
        )

        # ConversationSummaryBufferMemory will auto-summarize older turns.
        # Increase/decrease max_token_limit based on how much context you want retained.
        self.memory = ConversationSummaryBufferMemory(
            llm=self.summary_llm,
            chat_memory=self.history,
            return_messages=True,           # we’ll convert messages to OpenAI format
            input_key="input",
            output_key="output",
            max_token_limit=int(os.getenv("LC_MAX_TOKENS", "3000")),
        )

    # ---------------------- MCP wiring ----------------------
    def _openai_tools_from_mcp(self, mcp_tools: List[types.Tool]) -> List[dict]:
        tools = []
        for t in mcp_tools:
            tools.append({
                "type": "function",
                "function": {
                    "name": t.name,
                    "description": t.description or "",
                    "parameters": t.inputSchema or {"type": "object", "properties": {}},
                }
            })
        return tools

    async def connect_to_server(self, server_name: str, server_cfg: dict):
        """Connect to one MCP server (stdio)."""
        try:
            params = StdioServerParameters(**server_cfg)
            read, write = await self.exit_stack.enter_async_context(stdio_client(params))
            session = await self.exit_stack.enter_async_context(ClientSession(read, write))
            await session.initialize()

            self.sessions.append(session)

            resp = await session.list_tools()
            tools = resp.tools or []
            print(f"Connected to {server_name} with tools:", [t.name for t in tools])

            for t in tools:
                self.tool_to_session[t.name] = session
            self.available_tools.extend(self._openai_tools_from_mcp(tools))
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self, config_path: str = "server_config.json"):
        """Read server_config.json and connect to each server."""
        with open(config_path, "r", encoding="utf-8") as f:
            cfg = json.load(f)
        servers = cfg.get("mcpServers", {})
        for name, server_cfg in servers.items():
            await self.connect_to_server(name, server_cfg)

    # ---------------------- Memory helpers (NEW) ----------------------
    def _lc_history_to_openai_messages(self, history: Union[str, List[Any]]) -> List[Dict[str, str]]:
        """Convert LangChain memory 'history' to OpenAI Chat Completions messages."""
        msgs: List[Dict[str, str]] = []
        if isinstance(history, str):
            if history.strip():
                msgs.append({
                    "role": "system",
                    "content": f"Conversation summary so far:\n{history}"
                })
            return msgs

        # history is a list of BaseMessage
        for m in history:
            if isinstance(m, HumanMessage):
                msgs.append({"role": "user", "content": m.content})
            elif isinstance(m, AIMessage):
                # We store only the assistant's final content between turns.
                msgs.append({"role": "assistant", "content": m.content})
            elif isinstance(m, SystemMessage):
                msgs.append({"role": "system", "content": m.content})
            else:
                # Skip tool/other message types to keep things simple
                continue
        return msgs

    def _remember_turn(self, user_text: str, ai_text: str):
        """Append the latest user/assistant messages to LangChain memory."""
        self.memory.chat_memory.add_user_message(user_text)
        self.memory.chat_memory.add_ai_message(ai_text)

    def clear_memory(self):
        """Erase persisted memory (type 'forget' in the REPL)."""
        self.history.clear()
        print("✅ Memory cleared.")

    # ---------------------- Chat loop with memory ----------------------
    async def process_query(self, query: str, model: str = "gpt-3.5-turbo"):
        # 1) Pull prior memory (summarized or raw messages) and convert to OpenAI format
        prior = self.memory.load_memory_variables({}).get("history", [])
        messages: List[Dict[str, Any]] = self._lc_history_to_openai_messages(prior)

        # 2) Add the new user input
        messages.append({"role": "user", "content": query})

        # 3) Chat/tool loop
        while True:
            resp = self.client.chat.completions.create(
                model=model,
                messages=messages,
                tools=self.available_tools or None,
                tool_choice="auto" if self.available_tools else "none",
                temperature=0.1,
            )
            msg = resp.choices[0].message

            if msg.tool_calls:
                # Keep assistant msg (with tool_calls) in history for the model context
                messages.append({
                    "role": "assistant",
                    "content": msg.content or "",
                    "tool_calls": [tc.model_dump() for tc in msg.tool_calls],
                })

                for tc in msg.tool_calls:
                    tool_name = tc.function.name
                    raw_args = tc.function.arguments
                    try:
                        args = json.loads(raw_args) if isinstance(raw_args, str) else (raw_args or {})
                    except Exception:
                        args = {}

                    session = self.tool_to_session.get(tool_name)
                    if not session:
                        messages.append({
                            "role": "tool",
                            "tool_call_id": tc.id,
                            "name": tool_name,
                            "content": f"Tool '{tool_name}' is not available.",
                        })
                        continue

                    print(f"Calling tool {tool_name} with args {args}")
                    result = await session.call_tool(tool_name, arguments=args)
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tc.id,
                        "name": tool_name,
                        "content": _flatten_tool_content(result.content),
                    })
                continue  # let the model read tool results and continue

            # Final answer (no tool calls)
            final_text = (msg.content or "").strip()
            if final_text:
                print(final_text)

            # 4) Store this turn in memory (NEW)
            #    We store only user query and final assistant message for compactness.
            try:
                self._remember_turn(query, final_text if final_text else "[no text]")
            except Exception as mem_err:
                # Don’t crash chat if memory write fails
                print(f"(Memory error — continuing without memory this turn): {mem_err}")

            break

    async def chat_loop(self):
        print(f"\nMCP Chatbot (OpenAI {os.getenv('OPENAI_SUMMARY_MODEL')} + LangChain Memory) — type your query, 'forget' to clear memory, or 'quit' to exit.")
        while True:
            try:
                q = input("\nQuery: ").strip()
                if q.lower() == "quit":
                    break
                if q.lower() == "forget":
                    self.clear_memory()
                    continue
                await self.process_query(q)
            except Exception as e:
                print(f"Error: {e}")

    async def run(self):
        try:
            await self.connect_to_servers()
            await self.chat_loop()
        finally:
            await self.exit_stack.aclose()

    async def cleanup(self):  # existing
        """Cleanly close all resources using AsyncExitStack."""
        await self.exit_stack.aclose()


async def main():
    chatbot = MCP_ChatBot()
    try:
        await chatbot.connect_to_servers()  # init MCP servers
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup()


if __name__ == "__main__":
    asyncio.run(main())


Overwriting mcp_chatbot_openai.py


In [30]:
%%writefile mcp_chatbot.py
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List, Dict, TypedDict, Any, Optional, Union
from contextlib import AsyncExitStack
import asyncio, json, os, requests

# ===== LangChain imports (local summarizer + persistent history) =====
from langchain_ollama import ChatOllama
from langchain.memory import ConversationSummaryBufferMemory   # <- use this
from langchain_community.chat_message_histories import FileChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

load_dotenv()

# ---------------------------
# Ollama adapter (no OpenAI)
# ---------------------------
class OllamaAdapter:
    def __init__(self, base: str = "http://127.0.0.1:11434"):
        self.base = base.rstrip("/")

    def chat_once(
        self,
        model: str,
        messages: List[Dict[str, Any]],
        temperature: float = 0.1,
        tools: Optional[List[dict]] = None,
        tool_choice: str | dict | None = "auto",
    ):
        payload = {
            "model": model,
            "messages": messages,
            "stream": False,
            "options": {"temperature": temperature},
        }
        if tools:
            payload["tools"] = tools
        if tool_choice:
            payload["tool_choice"] = tool_choice

        r = requests.post(f"{self.base}/api/chat", json=payload, timeout=120)
        r.raise_for_status()
        data = r.json()
        msg = data.get("message", {}) or {}
        # Normalize into an OpenAI-like shape the rest of the code expects
        return {
            "choices": [{
                "message": {
                    "role": msg.get("role", "assistant"),
                    "content": msg.get("content", ""),
                    "tool_calls": msg.get("tool_calls") or [],
                }
            }]
        }

def _flatten_tool_content(content_list) -> str:
    parts = []
    if isinstance(content_list, list):
        for item in content_list:
            t = getattr(item, "type", None) or (isinstance(item, dict) and item.get("type"))
            if t == "text":
                txt = getattr(item, "text", None) or (isinstance(item, dict) and item.get("text"))
                if txt is not None:
                    parts.append(str(txt))
            elif t in ("json", "object"):
                data = getattr(item, "data", None) or (isinstance(item, dict) and (item.get("data") or item.get("value")))
                try:
                    parts.append(json.dumps(data, ensure_ascii=False))
                except Exception:
                    parts.append(str(data))
            else:
                try:
                    parts.append(json.dumps(item, default=str, ensure_ascii=False))
                except Exception:
                    parts.append(str(item))
    elif content_list is not None:
        if isinstance(content_list, (dict, list)):
            parts.append(json.dumps(content_list, ensure_ascii=False))
        else:
            parts.append(str(content_list))
    return "\n".join(parts).strip()

class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict

class MCP_ChatBot:
    def __init__(self):
        # ===== Models & endpoints =====
        self.model_default = os.getenv("LOCAL_MODEL", "qwen2.5:1.5b")
        self.client = OllamaAdapter(os.getenv("LOCAL_BASE_URL", "http://127.0.0.1:11434"))

        # ===== MCP infra =====
        self.exit_stack = AsyncExitStack()
        self.sessions: List[ClientSession] = []
        self.tool_to_session: Dict[str, ClientSession] = {}
        self.available_tools: List[ToolDefinition] = []

        # ===== LangChain Memory (NEW) =====
        # 1) Persistent chat history file (override with LC_HISTORY_PATH)
        history_path = os.getenv("LC_HISTORY_PATH", ".mcp_chat_history.json")
        self.history = FileChatMessageHistory(history_path)

        # 2) Local summarizer model via Ollama (override with LC_SUMMARY_MODEL)
        #    Pick a small/fast model installed in Ollama (e.g., llama3.2:3b, qwen2.5:1.5b, phi4:latest).
        summary_model = os.getenv("LC_SUMMARY_MODEL", "qwen2.5:1.5b")
        self.summary_llm = ChatOllama(
            model=summary_model,
            base_url=os.getenv("LOCAL_BASE_URL", "http://127.0.0.1:11434"),
            temperature=0.0
        )

        # 3) Summary buffer to keep memory compact
        self.memory = ConversationSummaryBufferMemory(
            llm=self.summary_llm,
            chat_memory=self.history,          
            return_messages=True,
            input_key="input",
            output_key="output",
            max_token_limit=int(os.getenv("LC_MAX_TOKENS", "3000")),
        )

    # ---------------------- MCP wiring ----------------------
    def _openai_tools_from_mcp(self, mcp_tools: List[types.Tool]) -> List[dict]:
        out = []
        for t in mcp_tools:
            out.append({
                "type": "function",
                "function": {
                    "name": t.name,
                    "description": t.description or "",
                    "parameters": t.inputSchema or {"type": "object", "properties": {}},
                }
            })
        return out

    async def _call_mcp_tool(self, tool_name: str, args: dict) -> Optional[str]:
        session = self.tool_to_session.get(tool_name)
        if not session:
            return f"Tool '{tool_name}' is not registered."
        try:
            result = await session.call_tool(tool_name, arguments=args or {})
            return _flatten_tool_content(result.content) or "(empty result)"
        except Exception as e:
            return f"Tool '{tool_name}' failed: {e}"

    async def connect_to_server(self, server_name: str, server_cfg: dict):
        """Connect to one MCP server (stdio)."""
        try:
            params = StdioServerParameters(**server_cfg)
            read, write = await self.exit_stack.enter_async_context(stdio_client(params))
            session = await self.exit_stack.enter_async_context(ClientSession(read, write))
            await session.initialize()

            self.sessions.append(session)

            resp = await session.list_tools()
            tools = resp.tools or []
            print(f"Connected to {server_name} with tools:", [t.name for t in tools])

            for t in tools:
                self.tool_to_session[t.name] = session
            self.available_tools.extend(self._openai_tools_from_mcp(tools))
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self, config_path: str = "server_config.json"):
        """Read server_config.json and connect to each server."""
        if not os.path.exists(config_path):
            print(f"No {config_path} found — skipping MCP server connections.")
            return
        try:
            with open(config_path, "r", encoding="utf-8") as f:
                cfg = json.load(f)
        except Exception as e:
            print(f"Failed to read {config_path}: {e}")
            return

        for name, server_cfg in (cfg.get("mcpServers", {}) or {}).items():
            await self.connect_to_server(name, server_cfg)

    # ---------------------- Memory helpers (NEW) ----------------------
    def _lc_history_to_openai_messages(self, history: Union[str, List[Any]]) -> List[Dict[str, str]]:
        """Convert LangChain memory history into Ollama/OpenAI-style messages."""
        msgs: List[Dict[str, str]] = []
        # If ConversationSummaryBufferMemory returns a summary string
        if isinstance(history, str):
            if history.strip():
                msgs.append({"role": "system", "content": f"Conversation summary so far:\n{history}"})
            return msgs

        # Otherwise it's a list[BaseMessage]
        for m in history:
            if isinstance(m, HumanMessage):
                msgs.append({"role": "user", "content": m.content})
            elif isinstance(m, AIMessage):
                msgs.append({"role": "assistant", "content": m.content})
            elif isinstance(m, SystemMessage):
                msgs.append({"role": "system", "content": m.content})
            # Skip tool messages here; tool outputs are re-fed within the same turn
        return msgs

    def clear_memory(self):
        self.history.clear()
        print("✅ Memory cleared.")

    # ---------------------- Chat loop with memory ----------------------
    async def process_query(self, query: str, model: Optional[str] = None):
        model = model or self.model_default

        # 1) Pull prior memory and convert to messages
        prior = self.memory.load_memory_variables({}).get("history", [])
        memory_msgs = self._lc_history_to_openai_messages(prior)

        # 2) Optional system preamble advertising tools
        system_preamble = None
        if self.available_tools:
            tool_names = [t['function']['name'] for t in self.available_tools]
            system_preamble = {
                'role': 'system',
                'content': 'You can call functions to satisfy user requests. Available tools: ' + ', '.join(tool_names)
            }

        # 3) Build the conversation to send to the model
        messages: List[Dict[str, Any]] = []
        messages.extend(memory_msgs)
        if system_preamble:
            messages.append(system_preamble)
        messages.append({'role': 'user', 'content': query})

        # 4) Full tool loop until model stops calling tools
        final_text: str = ""
        while True:
            resp = self.client.chat_once(
                model=model,
                messages=messages,
                temperature=0.1,
                tools=self.available_tools or None,
                tool_choice='auto' if self.available_tools else 'none',
            )
            msg = resp['choices'][0]['message']
            tool_calls = msg.get('tool_calls') or []

            if tool_calls:
                # Keep the assistant step (with tool_calls) in the transcript for the model
                messages.append({
                    'role': msg.get('role', 'assistant'),
                    'content': msg.get('content', '') or "",
                    'tool_calls': tool_calls,
                })

                for tc in tool_calls:
                    fn = tc.get('function', {}) or {}
                    name = fn.get('name')
                    if not name:
                        continue
                    raw_args = fn.get('arguments') or '{}'
                    try:
                        args = json.loads(raw_args) if isinstance(raw_args, str) else (raw_args or {})
                    except Exception:
                        args = {}

                    print(f"Calling tool {name} with args {args}")
                    result_text = await self._call_mcp_tool(name, args)

                    messages.append({
                        'role': 'tool',
                        'tool_call_id': tc.get('id') or name,  # Ollama may omit id
                        'name': name,
                        'content': result_text,
                    })
                # loop to let the model see tool outputs
                continue

            # No more tool calls — print final content if present and exit
            final_text = (msg.get('content') or "").strip()
            if final_text:
                print(final_text)
            break

        # 5) Save this turn into memory (triggers summarization as needed)
        try:
            self.memory.save_context({"input": query}, {"output": final_text or "[no text]"})
        except Exception as mem_err:
            print(f"(Memory error — continuing without memory this turn): {mem_err}")

    async def chat_loop(self):
        print(f"\nLocal MCP Chatbot (Ollama {self.model_default} + LangChain Memory) — type your query, 'forget' to clear memory, or 'quit' to exit.")
        while True:
            try:
                q = input("\nQuery: ").strip()
                if not q:
                    continue
                if q.lower() in ("quit", "exit"):
                    break
                if q.lower() == "forget":
                    self.clear_memory()
                    continue
                await self.process_query(q)
            except (EOFError, KeyboardInterrupt):
                break
            except Exception as e:
                print(f"Error: {e}")

    async def cleanup(self):
        # Ensure all sessions and the stdio pipes are closed
        try:
            await self.exit_stack.aclose()
        except Exception:
            pass

async def main():
    bot = MCP_ChatBot()
    try:
        await bot.connect_to_servers()  # discover and register tools
        await bot.chat_loop()
    finally:
        await bot.cleanup()

# Safe runner: works in normal terminals and in environments with an active event loop
def _run_async(coro):
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)
    else:
        return asyncio.create_task(coro)

if __name__ == "__main__":
    _run_async(main())


Overwriting mcp_chatbot.py


In [22]:
from ollama import Client
client = Client(host="http://127.0.0.1:11434")

# Non-streaming
res = client.chat(model="qwen2.5:7b", messages=[{"role":"user","content":"Hello from the Ollama client."}])
print(res["message"]["content"])

# Streaming
for part in client.chat(model="qwen2.5:7b",
                        messages=[{"role":"user","content":"Stream, please."}],
                        stream=True):
    print(part["message"]["content"], end="", flush=True)


Hello! It's nice to meet you through the Ollama client. How can I assist you today? Whether you have questions, need information, or just want to chat, feel free to let me know!
Sure, I can stream text for you! What would you like me to generate or discuss? Do you have any specific topic in mind?